In [1]:
import cobra,rba,lxml
from lxml import etree
import pandas as pd
from collections import OrderedDict

In [2]:
rtGSM = cobra.io.load_json_model('./gsmodel/iRhtoC_r2_2.json')
gs_excl = [g.id for g in rtGSM.genes if g.id[:2] != 'rt']

Academic license - for non-commercial use only


In [3]:
proteinsET = etree.Element('RBAProteins')

elems = ['listOfComponents', 'listOfMacromolecules']
for e in elems:
    proteinsET.append(etree.Element(e))

In [4]:
aas = 'ACDEFGHIKLMNPQRSTVWY'
aas = [i for i in aas]

comET = proteinsET.find('listOfComponents')
for aa in aas:
    att_dict = OrderedDict({'id':'COM-' + aa, 'type': 'amino_acid', 'weight':'1'})
    comET.append(etree.Element('component', attrib=att_dict))

In [5]:
df_genes = pd.read_excel('./data/genes_info_compilation_with_mito.xlsx', sheet_name='genes')
df_genes.index = df_genes.gene_id.to_list()

In [6]:
macET = proteinsET.find('listOfMacromolecules')
for g in rtGSM.genes:
    if g.id in gs_excl:
        continue
    
    att_dict = OrderedDict({'id':'MAC-' + g.id, 'compartment': 'PART-c'})
    macE = etree.SubElement(macET, 'macromolecule', attrib=att_dict)
    
    prot_seq = df_genes.sequence[g.id]
    macE_com = etree.SubElement(macE, 'composition')
    for aa in aas:
        att_dict = {'component':'COM-'+aa, 'stoichiometry':str(prot_seq.count(aa))}
        macE_com.append(etree.Element('componentReference', attrib=att_dict))

In [7]:
with open('./proteins.xml', 'wb') as f:
    f.write(etree.tostring(proteinsET, pretty_print=True))